# ANALISIS - GRUPO 5

Este notebook integra ambas estrategias:
- Usa `pipeline_simple.py` para regenerar el CSV limpio desde los .xlsx originales
- O bien, carga directamente `base_pampeana_analizada.csv` si ya existe


In [ ]:
import os
import pandas as pd
from pipeline_simple import construir_base_limpia

# === Config ===
PERSONAS = "usu_individual_T324.xlsx"
HOGARES = "usu_hogar_T324.xlsx"
OUT_CSV = "base_pampeana_analizada.csv"

# === Cargar datos ===
if os.path.exists(OUT_CSV):
    print("[INFO] Cargando base limpia desde CSV existente…")
    df = pd.read_csv(OUT_CSV)
else:
    print("[INFO] Generando CSV limpio desde los archivos .xlsx…")
    df = construir_base_limpia(PERSONAS, HOGARES, OUT_CSV)

# === Visualizar muestra ===
df.sample(5)

# === Ver resumen de ingreso ===
df['ingreso'].describe()

# === Agregar variable log_ingreso para modelado ===
df['log_ingreso'] = np.log(df['ingreso'])

# === Exportar para análisis posterior (opcional) ===
df.to_csv("eph_final_analizada.csv", index=False)
print("[OK] Base exportada como eph_final_analizada.csv")
